In [ ]:
pip install tensorflow

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from skimage.util import random_noise
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, UpSampling2D
from PIL import Image
# Load your dataset
# You can replace 'dataset_url' with the URL of your dataset
# dataset_url = 'https://www.example.com/dataset.zip'
# Add code here to download and load your dataset

# Preprocess your dataset
# Add code here to preprocess your dataset, such as normalization or resizing
from google.colab import drive
drive.mount('/content/drive')
dataset = pd.read_csv('/content/drive/My Drive/DL Lab/LAB-2/2_Dataset/Train1.csv')  # Replace 'kaggle_dataset.csv' with the actual dataset file

# Preprocess the dataset
# Assuming the dataset has 'image_path' column for image file paths and 'age_group' column for age group labels


# Define the input image dimensions
image_height = 128
image_width = 128
num_channels = 3

# Load and preprocess images
X = []
for image_path in '/content/drive/My Drive/DL Lab/LAB-2/2_Dataset/'+dataset['ID']:
    img = Image.open(image_path)
    img = img.resize((image_width, image_height))
    img = np.array(img)
    X.append(img)

# Split dataset into images and labels
X = np.array(X)

clean_images=X

# Split your dataset into training and testing sets
# Add code here to split your dataset into train_images and test_images

# Add noise to the clean images to create noisy images
noise_std = 0.1  # Standard deviation of the Gaussian noise
noisy_images = random_noise(clean_images, var=noise_std**2)

# Split the dataset into training and testing sets
train_images, test_images, train_noisy, test_noisy = train_test_split(clean_images, noisy_images, test_size=0.2, random_state=42)

# Preprocessing
# Assuming you want to normalize the pixel values between 0 and 1

# Convert the images to floating-point values between 0 and 1
#train_images = train_images.astype('float32') / 255.0
#test_images = test_images.astype('float32') / 255.0
#train_noisy = train_noisy.astype('float32') / 255.0
#test_noisy = test_noisy.astype('float32') / 255.0

# Display example images before and after preprocessing
fig, axes = plt.subplots(2, 2, figsize=(8, 8))
axes[0, 0].imshow(clean_images[0], cmap='gray')
axes[0, 0].set_title('Clean Image')
axes[0, 0].axis('off')
axes[0, 1].imshow(noisy_images[0], cmap='gray')
axes[0, 1].set_title('Noisy Image')
axes[0, 1].axis('off')
axes[1, 0].imshow(train_images[0], cmap='gray')
axes[1, 0].set_title('Preprocessed Clean Image')
axes[1, 0].axis('off')
axes[1, 1].imshow(train_noisy[0], cmap='gray')
axes[1, 1].set_title('Preprocessed Noisy Image')
axes[1, 1].axis('off')
plt.tight_layout()
plt.show()


# Add Gaussian noise to the images (simulating noisy images)
#noise_factor = 0.5
#train_noisy = train_images + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=train_images.shape)
#test_noisy = test_images + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=test_images.shape)

# Clip the pixel values between 0 and 1
#train_noisy = np.clip(train_noisy, 0., 1.)
#test_noisy = np.clip(test_noisy, 0., 1.)

# Define the CNN model architecture

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=train_noisy.shape[1:]))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(1, (3, 3), activation='sigmoid', padding='same'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(train_noisy, train_images, epochs=10, batch_size=128, validation_data=(test_noisy, test_images))

# Denoise test images
denoised_images = model.predict(test_noisy)

# Visualize the results
n = 10  # Number of images to display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original noisy images
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display denoised images
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(denoised_images[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

ModuleNotFoundError: No module named 'tensorflow'

In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from PIL import Image

# Load the dataset
dataset = pd.read_csv('/content/drive/MyDrive/DL Lab/LAB-2/2_Dataset/Train1.csv')  # Adjust path to your dataset

# Preprocess the dataset
# Assuming the dataset has 'ID' column for image file paths and 'Class' column for labels

# Define the input image dimensions
image_height = 128
image_width = 128
num_channels = 3

# Load and preprocess images
X = []
for image_path in dataset['ID']:  # Corrected iteration over image paths
    img = Image.open('/content/drive/MyDrive/DL Lab/LAB-2/2_Dataset/' + image_path)  # Adjust path concatenation
    img = img.resize((image_width, image_height))
    img = np.array(img)
    X.append(img)

# Split dataset into images and labels
X = np.array(X)
y = np.array(dataset['Class'])

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the number of classes
num_classes = len(np.unique(y))
le=LabelEncoder()
y_train=le.fit_transform(y_train)
y_test=le.fit_transform(y_test)
# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Define the model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, num_channels)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

# Train the model
batch_size = 32
epochs = 10
steps_per_epoch = len(X_train) // batch_size
model.fit(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=steps_per_epoch, epochs=epochs)

# Evaluate the model
accuracy = model.evaluate(X_test, y_test)[1]
print("Accuracy:", accuracy)

# Save the trained model
model.save('/content/drive/MyDrive/DL Lab/LAB-2/trained_model_NEW.h5')  # Adjust save path and file name


ModuleNotFoundError: No module named 'keras'

In [15]:
#@title
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from PIL import Image
from sklearn.preprocessing import LabelEncoder

# Load the dataset
dataset = pd.read_csv('/content/drive/MyDrive/DL Lab/LAB-2/2_Dataset/Train1.csv')  # Replace 'kaggle_dataset.csv' with the actual dataset file

# Preprocess the dataset
# Assuming the dataset has 'image_path' column for image file paths and 'age_group' column for age group labels


# Define the input image dimensions
image_height = 128
image_width = 128
num_channels = 3

# Load and preprocess images
X = []
for image_path in '/content/drive/MyDrive/DL Lab/LAB-2/2_Dataset/'+dataset['ID']:
    print(image_path)
    img = Image.open(image_path)
    img.show()
    img = img.resize((image_width, image_height))
    img = np.array(img)
    X.append(img)

# Split dataset into images and labels
X = np.array(X)

y = np.array(dataset['Class'])

# Encode labels to integers
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the number of classes
num_classes = len(label_encoder.classes_)


# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Define the input image dimensions
image_height = 128
image_width = 128
num_channels = 3

# Define the model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, num_channels)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

# Train the model
batch_size = 32
epochs = 20
steps_per_epoch = len(X_train) // batch_size
model.fit(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=steps_per_epoch, epochs=epochs)

# Evaluate the model
accuracy = model.evaluate(X_test, y_test)[1]
print("Accuracy:", accuracy)


# Save the trained model
model.save('/content/drive/MyDrive/trained_model_NEW_2_2_Dataset.h5')  # Replace 'trained_model.h5' with the desired file name

/content/drive/MyDrive/DL Lab/LAB-2/2_Dataset/0.jpg
/content/drive/MyDrive/DL Lab/LAB-2/2_Dataset/1.jpg
/content/drive/MyDrive/DL Lab/LAB-2/2_Dataset/3.jpg
/content/drive/MyDrive/DL Lab/LAB-2/2_Dataset/4.jpg
/content/drive/MyDrive/DL Lab/LAB-2/2_Dataset/5.jpg
/content/drive/MyDrive/DL Lab/LAB-2/2_Dataset/8.jpg
/content/drive/MyDrive/DL Lab/LAB-2/2_Dataset/9.jpg
/content/drive/MyDrive/DL Lab/LAB-2/2_Dataset/11.jpg
/content/drive/MyDrive/DL Lab/LAB-2/2_Dataset/12.jpg
/content/drive/MyDrive/DL Lab/LAB-2/2_Dataset/13.jpg
/content/drive/MyDrive/DL Lab/LAB-2/2_Dataset/15.jpg
/content/drive/MyDrive/DL Lab/LAB-2/2_Dataset/17.jpg
/content/drive/MyDrive/DL Lab/LAB-2/2_Dataset/19.jpg
/content/drive/MyDrive/DL Lab/LAB-2/2_Dataset/21.jpg
/content/drive/MyDrive/DL Lab/LAB-2/2_Dataset/22.jpg
/content/drive/MyDrive/DL Lab/LAB-2/2_Dataset/23.jpg
/content/drive/MyDrive/DL Lab/LAB-2/2_Dataset/24.jpg
/content/drive/MyDrive/DL Lab/LAB-2/2_Dataset/25.jpg
/content/drive/MyDrive/DL Lab/LAB-2/2_Dataset/26.jpg


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'